## Application of EMIT Data: Agricultural Lands
### South Platte and Republican River Basins, CO, USA

The South Platte basin is the most populus river basin in Colorado with more than 70% of the state's population living within this region (**[Colorado Water Plan Version 2; South Platte/Metro](https://dnrweblink.state.co.us/cwcbsearch/0/edoc/216720/South_Platte-Metro_BIP_Volume2_2022.pdf)**). This region also has the greatest concentration of irrigated lands in the state.

Add introduction here.

![Water Figure 14](https://raw.githubusercontent.com/CU-ESIIL/HYR-SENSE/main/images/water_fig14.png)
Image source: https://cwcb.colorado.gov/colorado-water-plan/technical-update-to-the-plan

## Search and Download EMIT Data Products

In this notebook, we will use the 'earthaccess' Python package to search and download EMIT scenes which overlap agricultural lands in the South Platte river basin. Using this package, we can search for EMIT scenes using a region of interest and identify data granules which meet certain criteria for analysis (i.e., cloud cover, seasonality, etc). 

### Step 1. Setup notebook

First, we need to import packages and set up some environment variables for the notebook.

In [ ]:
# Import modules 

import os, time, shutil
import folium
import earthaccess
import warnings
import folium.plugins
import pandas as pd
import geopandas as gpd
import rioxarray as rxr
import rasterio as rio
import numpy as np
import math

from rasterio.plot import show
from branca.element import Figure
from IPython.display import display
from shapely import geometry
from skimage import io
from datetime import timedelta
from shapely.geometry.polygon import orient
from matplotlib import pyplot as plt

dest = os.path.join(os.path.expanduser("~"),"HYR-SENSE","data","Agriculture")
# Define some projections
albers = 'EPSG:5070' # Albers Equal Area CONUS
llwgs84 = 'EPSG:4326' # Geographic - Latitude/Longitude WGS84

# Create backup copy of the ortho images on your data store on iplant?
create_backup = True # True/False.

print("Success!")

#### Transfer project data from the cloud

We want to copy over data stored in CyVerse so that we can access files quickly for analysis. To do this, we will use Python "shutil" package to copy files from the CyVerse data store to a local, temporary location in the HYR-SENSE GitHub repository. The data we are copying is stored in a shared ESIIL / HYR-SENSE location. We can copy the files just from the "Agriculture" module which contains data for this exercise. However, if you want to work with other modules, they can be accessed here as well.

In [ ]:
"""
Copy the data-store to 'local' directory
This enables quick access to data files
"""

# Identify the location of the HYR-SENSE "data store"
data_store_path = '/data-store/iplant/home/shared/esiil/HYR_SENSE/data/Agriculture'
# Set a destination path (this is a 'local' and temporary path)
dest = '/home/jovyan/HYR-SENSE/data/Agriculture/' # in the GitHub repo we cloned
if not os.path.exists(dest):
    os.mkdir(dest) # create the directory for the copied data, if needed
    
# Using 'shutil' package, copy all the files over
shutil.copytree(data_store_path, dest, dirs_exist_ok=True)

### Step 2. Load Data

#### Region of Interest: South Platte & Republican River Basins

Let's start by loading the boundary data for the South Platte river basin. This data comes from the Watershed Boundary Dataset (WBD) Hydrologic Unit Codes (HUC). Specifically, we are loading the HUC6 South Platte boundary.

In [ ]:
# Region of interest
roi = os.path.join(dest,'WBD_HUC6_SouthPlatte_Republican.gpkg')
#roi_wgs84 = gpd.read_file(roi).to_crs(llwgs84)
roi = gpd.read_file(roi).to_crs(albers)
print(roi.crs)

# Plot the ROI data
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
# Use geographic coordinates
roi.to_crs(4326).plot(ax=ax, color='white', edgecolor='black')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('South Platte and Republican River Basins (HUC6)')
plt.show()

### Agricultural land use data

In order to get a better picture of where the agricultural land is within the basin, we will load two datasets from the National Agricultural Statistics Service (NASS); 1 - the Cropland Data Layer (CDL) cultivated lands mask, and 2 - the CDL crop type data. To learn more about these data, visit https://croplandcros.scinet.usda.gov.

In [ ]:
# NASS CDL (ca. 2023)
cdl = os.path.join(dest,'CDL_2023_CO_SouthPlatte_Republican.tif')
cdl = rxr.open_rasterio(cdl, mask=True, cache=False).squeeze().astype(rio.uint8)
cdl = cdl.where(cdl != 0, np.nan)  # handle NA values
print(cdl)
print(cdl.rio.crs) # Check to make sure the projection matches (EPSG:5070)
# cdl = cdl.rio.clip(roi.geometry) # clip to the roi for consistency

In [ ]:
# Plot the ROI data
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
roi.plot(ax=ax, color='white', edgecolor='black', linewidth=5.0)
cdl.plot(ax=ax, add_colorbar=False, cmap='viridis')  # Customize the colormap as needed
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Cultivated Lands (NASS CDL 2023)')
plt.show()

## Search for EMIT Data

Using the 'earthaccess' Python package, we can search for EMIT data which overlaps our region of interest. In order to identify data which we can use for analysis, we will also filter for things like season and cloud cover using the metadata for EMIT granules. 

In [ ]:
import earthaccess 

# Log in to your NASA Earthdata
# You only have to do this once in your session (persist=True saves the credentials)
earthaccess.login(persist=True)

Now we can use the search functionality of 'earthaccess' to identify data products related to the EMIT mission. This code chunk will return "short names" of the data products available. To get a better idea of what each of these products are, you can do a quick search using these short names. For example, to read more about the L2A Estimated Surface Reflectance and Uncertainty and Masks 60-m product you can visit: https://lpdaac.usgs.gov/products/emitl2arflv001/ which pops up when searching the short name "EMITL2ARFL".

In [ ]:
# Query the collections
Query = earthaccess.collection_query().keyword('emit').provider('LPCLOUD')
print(f'Collections found: {Query.hits()}')
# Return EMIT specific products
collections = Query.fields(['ShortName']).get(Query.hits())
# Retrieve Collection Short-names
[product['short-name'] for product in [collection.summary() for collection in collections]]



To use a region of interest (ROI) polygon to search for EMIT data, we need to extract the coordinate pairs from our ROI (South Platte) which are then used in the search criteria. The code below will work for a single polygon or for multiple polygons if you wanted to search for data across a variety of ROIs. In this example, we use just one ROI, the South Platte River Basin.

In [ ]:
# ~ Extract a list coordinate pairs for ROI ~

# First, reproject to WGS84 for coordinates are in lat/long
roi = roi.to_crs(epsg=4326)

# Extract the bounds
coord_list = []
for iter, row in roi.iterrows():    
    bounds = row['geometry'].bounds
    coord_list.append(bounds)
print(coord_list)

Now we can run a search for EMIT L2A Reflectance data (short name = 'EMITL2ARFL'). To start, we will search for EMIT data during the **2023 growing season (May-Sept)**. The results from our search is a list of granules which meet the criteria. These granules have associated metadata and we can convert the search results to a data frame to access the granule attributes.

In [ ]:
# Search for EMIT data
search_results = earthaccess.search_data(
    short_name='EMITL2ARFL',
    bounding_box=coord_list[0], # we use [0] here because we only have one coordinate pair
    temporal=('2023-05-01','2023-09-28'),
    count=150
)
# Extract the metadata json as data frame
search_df = pd.json_normalize(search_results)
search_df.head()

In [ ]:
print("Total number of granules found: " + str(len(search_df)))

You can see above that we have identified 112 separate granules that meet our current search requirements

In [ ]:
# Get a list of columns from the search results / metadata
print(search_df.columns)

While the list suggests we have 112 granules, we need to apply some additional requirements to ensure the datasets are useable for our application. For example, clouds and cloud cover can impact any analyses looking at the land surface and we often want to "screen out" or filter out granules that have substantial cloud cover impacting our region of interest


## Filter EMIT granules and select 

In this section we will apply additional data filters to find granules that meet our research requirements. In this case we will select those with low cloud cover

In [ ]:
# Find cloud-free granules
print("Cloud cover statistics for granules: \n")
print(search_df['umm.CloudCover'].describe())
print("~~~~~~~~~~~~~~")

# How many granules have less than 5% cloud cover?
no_clouds_n = (search_df['umm.CloudCover'] <= 5).sum()
print(f"Number of granules with less than or equal to 5% cloud cover: {no_clouds_n}")
print("~~~~~~~~~~~~~~")

# Filter our search results to just these low cloud granules
search_df_nc = search_df[search_df['umm.CloudCover'] <= 5]

After searching for relatively cloud free images, we now see that the total number drops to 34.

Now we have a data frame with EMIT granules during the growing season with 0% cloud cover. To further refine this list, lets take a look at the dates of the granules. 

In [ ]:
# Print the acquisition dates for our no cloud granules
print(search_df_nc['umm.TemporalExtent.RangeDateTime.BeginningDateTime'].unique())

It looks like we have some good options for EMIT granules! To get a better idea of the spatial extent of these granules, we can plot their footprints on a map using the metadata from the data search results.

Thankfully, the metadata data frame contains information about the spatial extent for each granule which allows us to plot them on an interactive map using 'folium'. We can extract the spatial information and convert that to a geometry with the custom function below "get_shapely_object". This come from: XXX.

**In the next step we are defining two new custom functions that will help us evaluate which data granules meet out needs**

In [ ]:
def get_shapely_object(result:earthaccess.results.DataGranule):
    """
    Retrieve geospatial information from ECOSTRESS granule footprints.
    This function allows us to retrieve the geographic coverage for each granule and plot it on a map.
    
    :param 'result:earthaccess.results.DataGranule': a single data granule from earthaccess data search
    """
    
    # Get Geometry Keys
    geo = result['umm']['SpatialExtent']['HorizontalSpatialDomain']['Geometry']
    keys = geo.keys()

    if 'BoundingRectangles' in keys:
        bounding_rectangle = geo['BoundingRectangles'][0]
        # Create bbox tuple
        bbox_coords = (bounding_rectangle['WestBoundingCoordinate'],bounding_rectangle['SouthBoundingCoordinate'],
                       bounding_rectangle['EastBoundingCoordinate'],bounding_rectangle['NorthBoundingCoordinate'])
        # Create shapely geometry from bbox
        shape = geometry.box(*bbox_coords, ccw=True)
    elif 'GPolygons' in keys:
        points = geo['GPolygons'][0]['Boundary']['Points']
        # Create shapely geometry from polygons
        shape = geometry.Polygon([[p['Longitude'],p['Latitude']] for p in points])
    else:
         raise ValueError('Provided result does not contain bounding boxes/polygons or is incompatible.')
    return(shape)


# Convert bounding coordinates to Folium-ready data for mapping
def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation
    Leaflet interactive maps require a specific format for coordinates, this function sets that up for a given bounding box.

    ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))

print("Ready to use function(s)!")

You can see above that the functions were successfully defined and now we can use them in the next step

In [ ]:
# Retrieve the bounding information

# Create shapely polygons for result
geometries = [get_shapely_object(search_results[index]) for index in search_df_nc.index.to_list()]
# Convert to GeoDataframe
results_gdf = gpd.GeoDataFrame(search_df_nc, geometry=geometries, crs="EPSG:4326")
# Ensure 'start_datetime' and other relevant datetime columns are strings
results_gdf['start_datetime'] = results_gdf['umm.TemporalExtent.RangeDateTime.BeginningDateTime'].astype(str)
results_gdf = results_gdf[['meta.native-id','start_datetime','geometry']]

# Create the interactive map using folium
fig = Figure(width="750px", height="375px")
map1 = folium.Map(tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}', attr='Google')
fig.add_child(map1)

# Plot Region of Interest
roi.drop(columns=['LOADDATE']).explore(
    popup=False,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="South Platte River Basin",
    m=map1
)

results_gdf.explore(
    "meta.native-id",
    categorical=True,
    tooltip=[
        "meta.native-id",
        "start_datetime",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="EMIT",
    m=map1,
    legend=False
)

map1.fit_bounds(bounds=convert_bounds(results_gdf.unary_union.bounds))
map1.add_child(folium.LayerControl())
display(fig)

Using the interactive map above, you can now review the remaining granules meeting our research and location requirements. Using the map you can scroll around, hover over specific granule outlines to see details, or click on specific image outlines to get the details.  You can also zoom in and out to review areas with multiple overlapping granules from different dates

## Select and download EMIT granules to use in the rest of the tutorial

Based on the results above we will select four granules from 2023-07-29 that represent a small stip of images covering a large fraction of the agricultural landscapoe in the Southwestern/Northeastern portion of Nebraska and Colorado

In [ ]:
# Lets define the 4 granules from 2023-07-29 to download for the tutorial
selected_granules = ['EMIT_L2A_RFL_001_20230729T205653_2321014_021',
                     'EMIT_L2A_RFL_001_20230729T205642_2321014_020',
                     'EMIT_L2A_RFL_001_20230729T205630_2321014_019',
                     'EMIT_L2A_RFL_001_20230729T205618_2321014_018']

# Filter our metadata, plot the selected granules
granules = search_df_nc[search_df_nc['meta.native-id'].isin(selected_granules)]
granules_gdf = results_gdf[results_gdf['meta.native-id'].isin(selected_granules)]

# Create the interactive map using folium
fig = Figure(width="750px", height="375px")
map1 = folium.Map(tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}', attr='Google')
fig.add_child(map1)

# Plot Region of Interest
roi.drop(columns=['LOADDATE']).explore(
    popup=False,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="South Platte River Basin",
    m=map1
)

granules_gdf.explore(
    "meta.native-id",
    categorical=True,
    tooltip=[
        "meta.native-id",
        "start_datetime",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="EMIT",
    m=map1,
    legend=False
)

map1.fit_bounds(bounds=convert_bounds(granules_gdf.unary_union.bounds))
map1.add_child(folium.LayerControl())
display(fig)

Note: You can see in the map above the specific granules we have selected to download and work with in the tutorials

### Now lets download the four selected 

We will use a simple loop to download the granules from EarthData below and provide us a verbose output showing the progress of the data downloading.  The first time you run this you will note that a new "EMIT" folder is created in "HYR-SENSE/data/Agriculture/" 

In [ ]:
# Download the scenes and store in your "local" working/scratch folder for faster I/O and data access in the remaining tutorials
# local relates to the path /home/jovyan/HYR-SENSE/data/

# Retrieve the download URL
granule_ids = [f"{granule}/{granule}.nc" for granule in granules['umm.GranuleUR']]

# Get Https Session using Earthdata Login Info
fs = earthaccess.get_fsspec_https_session()

# Base download URL
base_url = 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001'

# Define local storage location on CyVerse - default is HYR-SENSE/data/emit
loc_file_path = os.path.join('/home/jovyan/HYR-SENSE/data/Agriculture/emit/')
if not os.path.exists(loc_file_path):
    os.mkdir(loc_file_path)

# Download the data granules
t0 = time.time()

for granule_id in granule_ids:
    print(granule_id)
    print("Granule ID: " + granule_id.split('/')[1])
    granule_asset_id = granule_id.split('/')[1]
    out_path = loc_file_path + f'{granule_asset_id}'

    # Download the Granule Asset if it doesn't exist
    url = os.path.join(base_url,granule_id)
    if not os.path.isfile(out_path):
        fs.download(url, out_path)

print('Total time:', time.time() - t0)

Let's also save the geospatial footprints for our selected tiles. This will be useful in the preceding notebooks.

In [ ]:
out_fpath = os.path.join(dest,'emit_granule_footprints.gpkg')
granules_gdf.to_file(out_fpath)
print(f"EMIT footprints saved to {out_fpath}")

### OPTIONAL: If you only want to download a single granule, we can select the image centered on Yuma, Colorado from July 29th 2023

In [ ]:
selected_granules = ['EMIT_L2A_RFL_001_20230729T205630_2321014_019']

# Filter our metadata, plot the selected granules
granules = search_df_nc[search_df_nc['meta.native-id'].isin(selected_granules)]
granules_gdf = results_gdf[results_gdf['meta.native-id'].isin(selected_granules)]

# Create the interactive map using folium
fig = Figure(width="750px", height="375px")
map1 = folium.Map(tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}', attr='Google')
fig.add_child(map1)

# Plot Region of Interest
roi.drop(columns=['LOADDATE']).explore(
    popup=False,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="South Platte River Basin",
    m=map1
)

granules_gdf.explore(
    "meta.native-id",
    categorical=True,
    tooltip=[
        "meta.native-id",
        "start_datetime",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="EMIT",
    m=map1,
    legend=False
)

map1.fit_bounds(bounds=convert_bounds(granules_gdf.unary_union.bounds))
map1.add_child(folium.LayerControl())
display(fig)

print(granules_gdf)

### Download the EMIT_L2A_RFL_001_20230729T205630_2321014_019

Will only download if the file doesnt already exist.  If it already exists, it will skip downloading and print back the granule ID

In [ ]:
### Download the scenes, store in working/scratch folder for faster I/O and data access in the remaining tutorials
# Retrieve the download URL
granule_ids = [f"{granule}/{granule}.nc" for granule in granules['umm.GranuleUR']]

# Get Https Session using Earthdata Login Info
fs = earthaccess.get_fsspec_https_session()

# Base download URL
base_url = 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001'

# Define local storage location on CyVerse - default is HYR-SENSE/data/Agriculture/emit
loc_file_path = os.path.join('/home/jovyan/HYR-SENSE/data/Agriculture/emit/')
if not os.path.exists(loc_file_path):
    os.mkdir(loc_file_path)

# Download the data granules
t0 = time.time()

for granule_id in granule_ids:
    print(granule_id)
    print("Granule ID: " + granule_id.split('/')[1])
    granule_asset_id = granule_id.split('/')[1]
    out_path = loc_file_path + f'{granule_asset_id}'
    print(out_path)
    # Download the Granule Asset if it doesn't exist
    url = os.path.join(base_url,granule_id)
    if not os.path.isfile(out_path):
        fs.download(url, out_path)
    if os.path.isfile(out_path):
        print("*** Already downloaded ***")
        print(" ")

print('Total time:', time.time() - t0)

## OPTIONAL: Create a copy of the downloaded data in the workshop storage

Let's create a backup copy from what is in the current scratch folder for later use and to avoid having to download again

In [ ]:
# (Optional) Copy data to CyVerse data store for download
if create_backup:
    user_name = os.environ.get("IPLANT_USER")
    print("*** Iplant username: " + user_name)
    src_dir = os.path.join('/home/jovyan/HYR-SENSE/data/Agriculture/emit/')
    dest_dir = os.path.join('/data-store/iplant/home/',user_name,'emit')
    print("")
    print("")
    print("**** Copy complete!! ****")
else:
    print("")
    print("*** Skipping backup of ortho images ***")  

In [ ]:
os.listdir(dest_dir)